In [ ]:
import os

In [ ]:
username = os.environ.get("USER") or os.environ.get("USERNAME")
if username:
    print(f"Username: {username}")
else:
    print("Username not found in environment variables.")
    sys.exit()

In [ ]:
from pathlib import Path
cwd = str(Path.cwd())
print(f"PWD: {cwd}")
os.environ["PYTHONPATH"] += f":{cwd}"
os.environ["X509_USER_PROXY"] = f"{cwd}/voms_proxy.txt"
os.environ['XRD_REQUESTTIMEOUT'] = "300"
!echo $PYTHONPATH

In [ ]:
from dask_gateway import Gateway
# gateway = Gateway() # For SLURM backend
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
) # For KUBERNET backend

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [ ]:
cluster = gateway.new_cluster(
    # conda_env = "/depot/cms/kernels/coffea2024",
    # conda_env = "/depot/cms/kernels/root632",
    # conda_env = "/depot/cms/kernels/coffea_latest",
    # conda_env = "/depot/cms/kernels/coffea_dask",
    conda_env = "/depot/cms/users/yun79/conda_envs/yun_coffea_latest",
    # conda_env = "/depot/cms/users/yun79/conda_envs/pfn_env",
    worker_cores = 1, #2
    # worker_memory = 60, #6
    worker_memory = 55, #For data with 2500 files use 10GB workers
    env = dict(os.environ),
)

cluster.adapt(minimum=25, maximum=399) # 399 for low resources, otherwise 199
cluster
# cluster.adapt(100)

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [ ]:
ClusterID_ToShutdown = "dd7659b707d34e33a0506fbdb6593a3d"
for cluster_info in gateway.list_clusters():
    # print(cluster_info.name)
    if ClusterID_ToShutdown in cluster_info.name:
        print(cluster_info.name)
        gateway.connect(cluster_info.name).shutdown()


In [ ]:
"""
To shutdown the cluster, run this cell
Equivalently, you can also do "squeue -u <username>"
if this is the only slurm job you're doing
"""

for cluster_info in gateway.list_clusters():
    gateway.connect(cluster_info.name).shutdown()

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)